In [1]:
import models 
import datasets
import torch
import pytorch_lightning as pl
import torchmetrics
import sklearn
import sklearn.preprocessing
import pandas as pd
import numpy as np

/home/stc/rybin-as/miniconda3/envs/torch/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
train = pd.read_csv('train.csv')
train['transcript'] = train['transcript'].fillna('[непереводимаяиграслов]')
val = pd.read_csv('val.csv')
val['transcript'] = val['transcript'].fillna('[непереводимаяиграслов]')
le = sklearn.preprocessing.LabelEncoder()
le.fit(train['EmotionMaxVote'].values)

vectorizer = sklearn.feature_extraction.text.TfidfVectorizer(max_features=1000, stop_words='english')
vectorizer.fit(train['transcript'])

train_dataset = datasets.text_dataset(train['transcript'], train['EmotionMaxVote'], vectorizer=vectorizer)
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)

val_dataset = datasets.text_dataset(val['transcript'], val['EmotionMaxVote'], vectorizer=vectorizer)
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=32, shuffle=True)

print(len(train_dataset), len(val_dataset), len(le.classes_))

2442 617 7


In [4]:
model = models.tfidf_model(1000, 7, DR=0.5)
logger = pl.loggers.tensorboard.TensorBoardLogger("text", name="tfidf")

trainer = pl.Trainer(accelerator="gpu", devices=1, max_epochs=20, logger=logger)
trainer.fit(model, train_dataloader, val_dataloader)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Missing logger folder: text/tfidf
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type             | Params
-----------------------------------------------
0 | D1        | Linear           | 64.1 K
1 | D2        | Linear           | 4.2 K 
2 | D3        | Linear           | 455   
3 | drop      | Dropout          | 0     
4 | f1_train  | F1Score          | 0     
5 | f1_val    | F1Score          | 0     
6 | acc_train | Accuracy         | 0     
7 | acc_val   | Accuracy         | 0     
8 | loss      | CrossEntropyLoss | 0     
-----------------------------------------------
68.7 K    Trainable params
0         Non-trainable params
68.7 K    Total params
0.275     Total estimated model params size (MB)


/home/stc/rybin-as/miniconda3/envs/torch/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:240: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Epoch 19: 100%|██████████| 97/97 [00:54<00:00,  1.79it/s, loss=0.841, v_num=0] 


In [ ]:
model = models.lstm_model(1000, 7, DR=0.5)
logger = pl.loggers.tensorboard.TensorBoardLogger("text", name="lstm")

trainer = pl.Trainer(accelerator="gpu", devices=1, max_epochs=20, logger=logger)
trainer.fit(model, train_dataloader, val_dataloader)